In [667]:
import sys
import json
import argparse
import random
import csv
import numpy as np
from scipy.signal import argrelextrema
from pandas import read_csv
import random


import tensorflow as tf
import keras
import PIL
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization
def read_file(file_path, type):
    if type == "csv":
        data = read_csv(file_path)
        return data.values
    elif type == "json":
        f = open(file_path)
        data = json.load(f)
        return data

In [1153]:
session_path = 'toadstool/participants/participant_0/participant_0_session.json'
truth_path = 'truth/participant_0_events.json'
vtruth_path = 'vtruth/participant_0_video.csv'
yourPath = 'scripts/i0'
vt = read_file(vtruth_path, "csv")
inp0='emoinput/p0_emotion.csv'
in0 = read_file(inp0, "csv")

In [1154]:
   # event we have
    event = {0:"new_stage", 1:"flag_reached", 2:"status_up", 3:"status_down", 4:"life_lost"}
    out = {"new_stage":0, "flag_reached":1, "status_up":2, "status_down":3, "life_lost":4}
 # game frame number
    game = read_file(session_path, "json")#frame的每個數字
    game_len = int(game["obs_n"])

In [1155]:
 game_len

124460

In [1156]:
temotion = pd.DataFrame(vt)#temotion要先 

In [1157]:
temotion

,0,1,2,3,4,5,6,7,8
0,0,"(273, 269, 139, 139)",0.01,0,0.04,0.07,0.06,0,0.81
1,1,"(275, 270, 138, 138)",0.01,0,0.03,0.08,0.03,0,0.86
2,2,"(274, 272, 137, 137)",0.01,0,0.04,0.05,0.05,0,0.85
3,3,"(276, 273, 134, 134)",0.01,0,0.04,0.08,0.03,0,0.85
4,4,"(278, 275, 130, 130)",0,0,0.04,0.07,0.02,0,0.86
...,...,...,...,...,...,...,...,...,...
62935,62935,"(362, 134, 258, 258)",0.12,0,0.15,0.03,0.55,0,0.15
62936,62936,"(365, 131, 247, 247)",0.12,0,0.11,0.02,0.53,0,0.21
62937,62937,"(373, 150, 216, 216)",0.08,0,0.18,0.01,0.65,0,0.08
62938,62938,"(368, 145, 218, 218)",0.13,0,0.07,0.01,0.57,0,0.22


In [1158]:
temotion.shape[0]

62940

In [1159]:
# temotion.set_index(temotion[8],inplace=True)#原本
temotion = temotion.drop([0],axis=1)
temotion = temotion.drop([1],axis=1)#原本無
temotion

,2,3,4,5,6,7,8
0,0.01,0,0.04,0.07,0.06,0,0.81
1,0.01,0,0.03,0.08,0.03,0,0.86
2,0.01,0,0.04,0.05,0.05,0,0.85
3,0.01,0,0.04,0.08,0.03,0,0.85
4,0,0,0.04,0.07,0.02,0,0.86
...,...,...,...,...,...,...,...
62935,0.12,0,0.15,0.03,0.55,0,0.15
62936,0.12,0,0.11,0.02,0.53,0,0.21
62937,0.08,0,0.18,0.01,0.65,0,0.08
62938,0.13,0,0.07,0.01,0.57,0,0.22


In [1160]:
# Scale the data###temotion
from sklearn.preprocessing import MinMaxScaler
import numpy as ny
newframe = []
frame = list(range(temotion.shape[0]))
frame=np.array(frame)
frame = frame[:,np.newaxis]

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(frame)

for i in scaled:
    i=float(i*game_len)
    newframe.append(round(i))

In [1161]:
temotion

,2,3,4,5,6,7,8
0,0.01,0,0.04,0.07,0.06,0,0.81
1,0.01,0,0.03,0.08,0.03,0,0.86
2,0.01,0,0.04,0.05,0.05,0,0.85
3,0.01,0,0.04,0.08,0.03,0,0.85
4,0,0,0.04,0.07,0.02,0,0.86
...,...,...,...,...,...,...,...
62935,0.12,0,0.15,0.03,0.55,0,0.15
62936,0.12,0,0.11,0.02,0.53,0,0.21
62937,0.08,0,0.18,0.01,0.65,0,0.08
62938,0.13,0,0.07,0.01,0.57,0,0.22


In [1162]:
temotion['normalize'] = newframe
temotion.set_index(temotion['normalize'],inplace=True)
temotion = temotion.drop(['normalize'],axis=1)
#drop 3,7,8 disgust,surprise,neutral,box
temotion = temotion.drop([3],axis=1)
temotion = temotion.drop([6],axis=1)
temotion = temotion.drop([7],axis=1)
temotion = temotion.drop([8],axis=1)


temotion.columns = ['angry','fear','happy']

temotion

,angry,fear,happy
normalize,,,
0,0.01,0.04,0.07
2,0.01,0.03,0.08
4,0.01,0.04,0.05
6,0.01,0.04,0.08
8,0,0.04,0.07
...,...,...,...
124452,0.12,0.15,0.03
124454,0.12,0.11,0.02
124456,0.08,0.18,0.01


In [1163]:
# temotion['normalize'] = newframe##原本
# temotion.set_index(temotion['normalize'],inplace=True)
# temotion = temotion.drop(['normalize'],axis=1)
# #drop 2,3,7,8 angry,disgust,surprise,neutral,box
# temotion = temotion.drop([2],axis=1)
# temotion = temotion.drop([7],axis=1)
# temotion = temotion.drop([8],axis=1)
# temotion.columns = ['angry','fear','happy','sad','surprise']

# temotion

In [1164]:
#findmax and record
#這個是要跟圖片一起的數字
emomax = np.array(temotion)
e = []
for i in emomax:
    e.append(np.argmax(i))

In [1165]:
temotion['max']=e
temotion

,angry,fear,happy,max
normalize,,,,
0,0.01,0.04,0.07,2
2,0.01,0.03,0.08,2
4,0.01,0.04,0.05,2
6,0.01,0.04,0.08,2
8,0,0.04,0.07,2
...,...,...,...,...
124452,0.12,0.15,0.03,1
124454,0.12,0.11,0.02,0
124456,0.08,0.18,0.01,1


In [1166]:
tmp = temotion.drop(['angry'],axis=1)
# tmp = tmp.drop(['surprise'],axis=1)
tmp = tmp.drop(['max'],axis=1)
# # tmp = tmp.drop(['fear'],axis=1)
# tmp = tmp.drop(['sad'],axis=1)
tmp

,fear,happy
normalize,,
0,0.04,0.07
2,0.03,0.08
4,0.04,0.05
6,0.04,0.08
8,0.04,0.07
...,...,...
124452,0.15,0.03
124454,0.11,0.02
124456,0.18,0.01


In [1167]:
    # write output##go to l
    output_path='emotrain/tmp0.csv'
    tmp.to_csv(output_path, index = True)

In [1168]:
##l

path='emotrain/tmp0.csv'
a = read_file(path, "csv")
a = pd.DataFrame(a)
arr = np.array(a)
local_max_idx = argrelextrema(arr , np.greater)
l= list(local_max_idx[0])
line = []
j=1
for i in range(round(len(l)/10)-1):
    line.append(l[j])
    j = j+10
print(len(line))
l = set(line)
l = pd.DataFrame(l)
l

2442


,0
0,3
1,32771
2,49155
3,40966
4,57350
...,...
2437,57324
2438,40944
2439,24563
2440,8185


In [1169]:
# Scale the data###l start
from sklearn.preprocessing import MinMaxScaler
import numpy as ny
lframe = []
frame = list(range(l.shape[0]))
frame=np.array(frame)
frame = frame[:,np.newaxis]

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(frame)

for i in scaled:
    i=float(i*game_len)
    lframe.append(round(i))
l['normalize'] = lframe
l.set_index(l['normalize'],inplace=True)

l['cnt'] = list(range(len(l)))
l.set_index(l['cnt'],inplace=True)

l

,0,normalize,cnt
cnt,,,
0,3,0,0
1,32771,51,1
2,49155,102,2
3,40966,153,3
4,57350,204,4
...,...,...,...
2437,57324,124256,2437
2438,40944,124307,2438
2439,24563,124358,2439


In [1170]:
import os


allFileList = os.listdir(yourPath)
X_train = []
Y_train = []
T= lframe
P=[]

for i in T:
    if i==0:
        file='game_'+str(1)+'.png'
        p = yourPath + "/" +file
        P.append(p)
          
    else:
        file='game_'+str(i)+'.png'
        p = yourPath + "/" +file
    #             img = cv2.imread(p)
        P.append(p) 
l['img'] = P
l

,0,normalize,cnt,img
cnt,,,,
0,3,0,0,scripts/i0/game_1.png
1,32771,51,1,scripts/i0/game_51.png
2,49155,102,2,scripts/i0/game_102.png
3,40966,153,3,scripts/i0/game_153.png
4,57350,204,4,scripts/i0/game_204.png
...,...,...,...,...
2437,57324,124256,2437,scripts/i0/game_124256.png
2438,40944,124307,2438,scripts/i0/game_124307.png
2439,24563,124358,2439,scripts/i0/game_124358.png


In [1171]:
##back to 
temotion

,angry,fear,happy,max
normalize,,,,
0,0.01,0.04,0.07,2
2,0.01,0.03,0.08,2
4,0.01,0.04,0.05,2
6,0.01,0.04,0.08,2
8,0,0.04,0.07,2
...,...,...,...,...
124452,0.12,0.15,0.03,1
124454,0.12,0.11,0.02,0
124456,0.08,0.18,0.01,1


In [1172]:
def findnear(arr,val):
    arr = np.array(arr)
    idx = (np.abs(arr-val)).argmin()
    return arr[idx]

In [1173]:
farr = np.array(temotion.index)
A=l.index
emolist=[]
check=[]
for i in A:#選最近數字
        nearest = findnear(farr,i)
        arr = np.array(temotion.loc[nearest])#選temotion裡的frame
        emolist.append(arr)
        check.append(nearest)#427

In [1174]:
emodf = pd.DataFrame(emolist)
emodf.columns = ['angry','fear','happy','max']
emodf

,angry,fear,happy,max
0,0.01,0.04,0.07,2
1,0.01,0.04,0.07,2
2,0.01,0.03,0.08,2
3,0.01,0.03,0.08,2
4,0.01,0.04,0.05,2
...,...,...,...,...
2437,0.00,0.03,0.01,1
2438,0.00,0.04,0.01,1
2439,0.00,0.04,0.01,1
2440,0.00,0.03,0.01,1


In [1175]:
validation = pd.concat([l, emodf], axis = 1)
validation

,0,normalize,cnt,img,angry,fear,happy,max
0,3,0,0,scripts/i0/game_1.png,0.01,0.04,0.07,2
1,32771,51,1,scripts/i0/game_51.png,0.01,0.04,0.07,2
2,49155,102,2,scripts/i0/game_102.png,0.01,0.03,0.08,2
3,40966,153,3,scripts/i0/game_153.png,0.01,0.03,0.08,2
4,57350,204,4,scripts/i0/game_204.png,0.01,0.04,0.05,2
...,...,...,...,...,...,...,...,...
2437,57324,124256,2437,scripts/i0/game_124256.png,0.00,0.03,0.01,1
2438,40944,124307,2438,scripts/i0/game_124307.png,0.00,0.04,0.01,1
2439,24563,124358,2439,scripts/i0/game_124358.png,0.00,0.04,0.01,1
2440,8185,124409,2440,scripts/i0/game_124409.png,0.00,0.03,0.01,1


In [1176]:
X = validation['img']
image_list = []
for file_name in X:
    image_array = np.array(PIL.Image.open(file_name))
    resized_image_array = cv2.resize(image_array, (128, 128))
    image_list.append(resized_image_array)
train_images = np.array(image_list)

In [1177]:
vmax=np.array(validation['max'])
vmax=np.reshape(vmax,(1,len(validation),1,1))
vmax

array([[[[2]],

        [[2]],

        [[2]],

        ...,

        [[1]],

        [[1]],

        [[1]]]])

In [1178]:
plus=np.insert(train_images,0,values=vmax,axis=1)
X =plus

In [1179]:
X.shape

(2442, 129, 128, 3)

In [1180]:
model = keras.models.load_model('newmario.h5')

In [1181]:
prediction = model.predict(X)

In [1182]:
prediction

array([[0.05018008, 0.34761444, 0.50113493, 0.16808924, 0.9999542 ],
       [0.05021316, 0.34348828, 0.49669984, 0.16913587, 0.9999547 ],
       [0.03197634, 0.51125664, 0.69960016, 0.08444822, 0.99997497],
       ...,
       [0.00952023, 0.09144509, 0.38694662, 0.7615135 , 0.99999845],
       [0.06468806, 0.19018415, 0.05897045, 0.7029306 , 0.9995804 ],
       [0.03724283, 0.11874196, 0.80447483, 0.7951572 , 0.9999517 ]],
      dtype=float32)

In [1183]:
pre = pd.DataFrame(prediction)
pre

,0,1,2,3,4
0,0.050180,0.347614,0.501135,0.168089,0.999954
1,0.050213,0.343488,0.496700,0.169136,0.999955
2,0.031976,0.511257,0.699600,0.084448,0.999975
3,0.052835,0.475311,0.406594,0.223199,0.999965
4,0.016419,0.094323,0.857696,0.610430,0.999896
...,...,...,...,...,...
2437,0.008597,0.278285,0.507094,0.410153,0.999996
2438,0.034880,0.460287,0.761077,0.893137,0.999956
2439,0.009520,0.091445,0.386947,0.761513,0.999998
2440,0.064688,0.190184,0.058970,0.702931,0.999580


In [1184]:
prediction = np.array(prediction)
result = []
for i in prediction:
    if i[0]>=0.7:
        result.append(0)
    else:
        result.append(np.argmax(i))
len(result)

2442

In [1185]:
re=[]
for i in result:
    re.append(event[i])
validation['result']=re
validation

,0,normalize,cnt,img,angry,fear,happy,max,result
0,3,0,0,scripts/i0/game_1.png,0.01,0.04,0.07,2,life_lost
1,32771,51,1,scripts/i0/game_51.png,0.01,0.04,0.07,2,life_lost
2,49155,102,2,scripts/i0/game_102.png,0.01,0.03,0.08,2,life_lost
3,40966,153,3,scripts/i0/game_153.png,0.01,0.03,0.08,2,life_lost
4,57350,204,4,scripts/i0/game_204.png,0.01,0.04,0.05,2,life_lost
...,...,...,...,...,...,...,...,...,...
2437,57324,124256,2437,scripts/i0/game_124256.png,0.00,0.03,0.01,1,life_lost
2438,40944,124307,2438,scripts/i0/game_124307.png,0.00,0.04,0.01,1,life_lost
2439,24563,124358,2439,scripts/i0/game_124358.png,0.00,0.04,0.01,1,life_lost
2440,8185,124409,2440,scripts/i0/game_124409.png,0.00,0.03,0.01,1,life_lost


In [1186]:
final = pd.concat([validation['normalize'], validation['result']], axis = 1)
final

,normalize,result
0,0,life_lost
1,51,life_lost
2,102,life_lost
3,153,life_lost
4,204,life_lost
...,...,...
2437,124256,life_lost
2438,124307,life_lost
2439,124358,life_lost
2440,124409,life_lost


In [1187]:
    # write output
    output_path='run/participant_0_events.csv'
    final.to_csv(output_path, index = False,header=False)